# Plotting and Curve Fitting Real Data

*We will use various modules to plot, manipulate, and fit astronomical data.  In addition, a primer on random numbers concludes this part.*

   Section A: [Radio Recombination Lines in the Galactic Center](#sectionA)

   Section B: [Using Data from Bubbles in the Milky Way](#sectionB)

   Section C: [Zooinverse Galaxy Zoo Data](#sectionC)
   
   Section D: [Random Numbers](#sectionD)

***
<a id='sectionA'></a>
## A. Radio Recombination Lines in the Galactic Center

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/00/Center_of_the_Milky_Way_Galaxy_IV_%E2%80%93_Composite.jpg/1920px-Center_of_the_Milky_Way_Galaxy_IV_%E2%80%93_Composite.jpg" width = "800x">

The [center of the Milky Way Galaxy](https://en.wikipedia.org/wiki/Galactic_Center) is 27,000 light years from our solar system and harbors a [4 million solar mass super massive black hole](https://en.wikipedia.org/wiki/Sagittarius_A*).  Despite the presence of this black hole, [star formation appears to still be occurring](https://arxiv.org/abs/0905.2161).  Evidence of such star formation is available in the surrounding gas which appears ionized due to radiation from bright young stellar objects.

Ionized gas consists of a soup of protons and electrons that are continuously recombining (and reionizing).  When an electron recombines with a proton it typically starts in a highly energetic state rather than immediately landing in the ground state.  The electron then cascades down one energy level at a time until it finds itself in the ground state (and eventually gets reionized). 

However, each time the electron moves down one energy level a photon is emitted that we can observe (called recombination lines).  We will use data from such an observation to measure the velocity of the gas.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from astropy.io import ascii

The data we will use is of the [mini-spiral](https://aasnova.org/2017/06/23/exploring-the-minispiral-at-the-milky-ways-center/) of the Galactic Center.  The mini-spiral is a collection of ionized gas 'streamers' that are oribiting the super massive black hole.  The observation is from [ALMA](https://almascience.nrao.edu/), a giant array of 12-m antennas in Chile.


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/The_Atacama_Compact_Array.jpg/1920px-The_Atacama_Compact_Array.jpg" width = "800x">


`H30a_gc.txt` refers to the galactic center (gc) at a frequency (~230 GHz) which corresponds to the photon that is emitted from an electron moving down one (alpha) energy level in hydrogen (H) gas from n=30.

In [ ]:
# Open the file. You can use np.genfromtxt, ascii.read (Megan's preferred way), 
# or Pandas dataframes if familiar. If using Pandas, remember to ignore file comments
spectra = np.genfromtxt('data/h30a_gc.txt') # ascii.read('data/h30a_gc.txt') #pd.read_csv('data/h30a_gc.txt', delim_whitespace=True, comment='#', header=None) #

# To access the columns, use indexing for np.genfromtxt ( [:,0] for first column, [:, 1] for second ),
# or use the name of columns (if provided) when using ascii.read (which is why I prefer it!)

vels = spectra[:,0] # units are km/s
flux = spectra[:,1] # units are Janskys (Jy)
flux *= 1e3  #convert Jy to mJy for convenience later

In [ ]:
# Now we can plot the data. Fill in the '---' below!

f,ax = plt.subplots(1,1, figsize=(8,5))

ax.plot(---, ---,'k', linewidth=0.7)
ax.set_xlim(---, ---) 
ax.set_xlabel('Velocity (km/s)')
ax.set_ylabel('Flux Density (mJy)')

Note that you can use `step` to plot rather than `plot`, which can be a useful reminder for spectra as each y-value is effectively a count (in this case intensity) for the corresponding velocity (or wavelength) bin - similar to a histogram. Either are fine for visualizing though.

## Now we will want to fit the data with a curve.  The shape of the emission line (you can make a guess by eye) is the model we want to fit the data with. But wait! Before we can directly fit this line though we need to subtract out the 'continuum' - the flat flux density around the emission line.

Remember that the line emission we see is due to electrons changning energy levels that was captured by a proton.  Often times though a free electron will interact with a proton but not actually get captured.  When this happens the force between the two particles causes an acceleration in the electron which in turn causes a photon to be emitted.  This is called [Bremsstrahlung](https://en.wikipedia.org/wiki/Bremsstrahlung) or free-free emission because both particles are free before and after the interaction.

The continuum we see is (in part) a result of this free-free emission because it can emit at any frequency.  However, since it detracts from what we are interested in (the recombination line emission) we will want to subtract it out. This continuum can be approximately modelled as linear, particularily for the data we have.  The fitting routine we will use `scipy`'s [`curve_fit`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) to fit a linear model the flat region of data (the continuum), but with one catch: we must also <i>exclude</i> the emission line when fitting the continuum.

In [ ]:
# The curve_fit function requires we define our model as a callable function, so lets do that.
# In this case it's a linear fit, so we'll use y = m x + b.
# Thus, you will need your function to accept three variables (x, m, b).

def linModel(x, m, b):
    y = --- #fill in!
    return y

Now, using the above plot, which range of velocities do we want to fit the continuum to? 

In [ ]:
# lets go with -400<v<-150 AND 0<v<400
# we can use np.where to find the index where these velocities are

# use the '&' operand insde of the np.where argument to constrain the regions
contInd_reg1 = np.where((--- < vels) & (vels < ---))[0]
contInd_reg2 = np.where((--- < vels) & (vels < ---))[0]
contInd = np.concatenate((contInd_reg1, contInd_reg2))  #putting together the two 'regions'

# now lets input this into linModel
# make sure you are only selecting the above region
# Notice that curve_fit returns a tuple (of arrays).
fitCoeffs, fitCovariances = curve_fit(---, vels[---], flux[---])

bestfit_m = fitCoeffs[---] #Which index returned by fitCoeffs will be the best-fit slope?
bestfit_b = fitCoeffs[---] #Which index returned by fitCoeffs will be the best-fit y-intercept?

print(f'best-fit value for \'m\': {bestfit_m}')
print(f'best-fit value for \'b\': {bestfit_b}')

Now that we have modelled the continuum, we need to subract it from the real continuum so that we are left with data that contains the line emission only.

In [ ]:
# get the modelled continuum
contModel = linModel(flux, bestfit_m, bestfit_b)
# subtract it from the data:
contFree = flux - ---

# now another plot to make sure it looks OK
# same as above but use 'contFree' instead of flux
f,ax = plt.subplots(1,1, figsize=(8,5), dpi=100)

ax.step(vels, ---, 'k', lw=0.7)
plt.show()

Notice the continuum level now.  Hopefully its at zero now?  If so good, then we can get what we are actually after - the emission line.  Which as perviously mentioned we are going to model as a Gaussian.

If we assume that the zero level of the Gaussian is at 0 (which we can safely do now that we subtracted the continuum), we can model the spectrum with the expression:

$$ y = I_0 e^{-4\ln{\left(2\right)} \alpha^2}$$
where,
$$ \alpha = \frac{x-v_0}{w}. $$

Here $I_0$ is the peak intensity, $v_0$ is the position of the peak intensity, and $w$ is the full width at half maximum (FWHM).

Similar to the line fit, we will need to define a function for a Gaussian before using `curve_fit`.

In [ ]:
# Lets make a function describing a gaussian AS SHOWN ABOVE for curve_fit
def gauss(x, *params):
    peak, vel, fwhm = params
    a = ---
    y = ---
    return y

Last time we used `curve_fit` we were able to let the function use a default set of initial guesses.  However, this time the function is much more complicated so we will have to make a guess first.  Use our initial plot of the spectrum to make a guess of the fit parameters.

In [ ]:
# p0 should be a list (or tuple) with our intial guesses of the three parameters
p0 = np.array([---, ---, ---]) # what's your initial guess?

# now we can try to fit the data
fitCoeffs, fitCov = curve_fit(gauss, vels, contFree, p0=p0)

In [ ]:
# lets plot the data again along with the fit to see if things look OK

fig, ax = plt.subplots(1,1, figsize=(8,5), dpi=100)
ax.plot(vels, contFree, 'k', lw=0.7)
# and plot the Gaussian resulting from the best-fit parameters!
ax.plot(vels, gauss(vels, *fitCoeffs), 'r')
plt.show()

### The uncertainty in the model fit

The second element that `curve_fit` produced is the [covariance matrix](https://en.wikipedia.org/wiki/Covariance_matrix), which contains information about the uncertainty of the best fit values.

### The diagonals are the squares of the standard deviations of our fit parameters.  In this case, we have three because we had three fit parameters.

The off-diagonal elements tell us the "covariance," a measure of how the two variables vary together – in other words, the degree to which two variables are linearly associated.

In [ ]:
print(f'Covariance matrix:\n{fitCov}')

In [ ]:
# getting the results of each parameter
peak = fitCoeffs[0]
vel = fitCoeffs[1]
fwhm = fitCoeffs[2]

# get the standard deviation, a.k.a. sigma, for each parameter.
peakErr = np.sqrt(fitCov[0,0])
velErr = np.sqrt(fitCov[1,1])
fwhmErr = np.sqrt(fitCov[2,2])

print(f'The peak intensity is : {peak:5.3} +/-{peakErr:4.1} mJy') #Notice this notation is convenient
print(f'The center velocity is: {vel:5.3} +/-{velErr:4.1} km/s') #for printing things neatly - 
print(f'The FWHM is           : {fwhm:5.3} +/-{fwhmErr:4.1} km/s') #remember this for later!

The quality of the data (and the fit) can be visualized by plotting the residual.  This is simply the difference between the fit and the data.

In [ ]:
# subtract the fit from the data
residual = contFree - gauss(---, ---)

# get the standard deviation of the residual:
std = np.std(---)
rms = 'RMS: ' + str(std)

# and plot
f,ax = plt.subplots(figsize=(8,4))
ax.step(vels, residual,'k', linewidth=0.7)
ax.text(0.05,0.9,rms,transform=ax.transAxes) # print out the rms on the plot
# make sure to add labels


#### Exercise!
Load up a different spectrum, 'data/h30a_gc_2comp.txt'.  This spectrum contains multiple (two) emission lines.  Perform the same analysis as above to fit both Gaussian components.  Plot and and print your results.  You don't have to subtract the continuum as it has already been done for this spectrum.   (*Further challenge: Can you write your code to work for an arbitary number of emission lines?*)

In [ ]:
# use as many cells as you need.
spectra = np.genfromtxt('data/h30a_gc_2comp.txt')
vels, flux = spectra[:,0], spectra[:,1]*1e3

contIds1 = np.where(vels<---)[0]
contIds2 = np.where(vels>---)[0]
contIds = np.concatenate((contIds1, contIds2))
# Fit the flat (linear) continuum first
fitcont_params, fitcont_cov = curve_fit(---, vels[---], flux[---])
fitcont = linModel(---, *---)
contFree = flux - fitcont

# Now block off each emission line
line1 = np.where((vels < ---) & (vels > ---))[0]
line2 = np.where((vels < ---) & (vels > ---))[0]

# Insert first guesses for each line 
# (try plotting the raw data first if you need to visualize what the guesses should be!)
p0_1 = np.array([---, ---, ---])
fitgauss1_params, fitgauss1_cov = curve_fit(gauss, vels[line1], flux[line1], p0=p0_1)
fitted_1 = gauss(vels[line1], *---)
p0_2 = np.array([---, --- ,---])
fitgauss2_params, fitgauss2_cov = curve_fit(gauss, vels[line2], flux[line2], p0=p0_2)
fitted_2 = gauss(vels[line2], *---)

fig, ax = plt.subplots(1,1, figsize=(8,5), dpi=100)
ax.plot(vels, contFree, 'k', lw=0.7)
ax.plot(vels[line1], fitted_1, 'r', lw=0.7)
ax.plot(vels[line2], fitted_2, 'b', lw=0.7)
plt.show()

***
<a id='sectionB'></a>
## B. Using Data from Bubbles in the Milky Way

<img src="http://www.spitzer.caltech.edu/uploaded_files/images/0008/5977/sig12-002_Med.jpg" width = "800x">

Now we will look at “bubbles” in the Milky Way that have been classified thanks to citizen scientists as part of the Zooniverse project. 

What are these “bubbles”? 

“They are regions around young massive stars that are so bright that their light has caused a shock wave that affects the cloud around them, blowing a bubble which we can see in infrared light. The dark interior is where the shock has already passed by and the bright red/pink ring around it is where the shock is currently impacting the gas cloud. Most of the circular features (bubbles) in these images are produced by hot young stars, as winds and radiation from these young stars sweep up the surrounding gas and dust from which they formed (like a snowplow that compresses the snow in its path.) Sometimes, the swept-up material becomes dense enough for gravity to pull it together to form new stars.” -- Zooniverse Milky Way

Let's visualize where the <a href="https://www.milkywayproject.org/">Zooniverse MilkyWay Project</a> Bubbles are located in our <a href="https://www.amnh.org/var/ezflow_site/storage/images/media/amnh/images/explore/ology-images/astronomy/the-milky-way-galaxy/milky-way-galaxy-topview/4971857-3-eng-US/milky-way-galaxy-topview_full_990.jpg">Milky Way Galaxy</a>. 

In your data folder is <a href="data/MWbubbles.fits">MWbubbles.fits</a>. This file has the classification results for all the large bubbles discovered through the MilkyWay Project. 

For each of the 3744 bubbles, MWbubbles.fits provides the Galactic longitude and latitude (see figure below), radius, thickness, eccentricity, position angle, hit rate, dispersion on position, and hierarchy flag. 

We'll focus on visualizing the Bubbles' Galactic latitude and longitude.

#### What is Galactic Latitude and Longitude?
* On the left is an image showing Earth-based longitude and latitude. 
* On the right is Galactic longitude and latitude. Same idea, but for our whole Galaxy. 

<img style="float: left" src="http://upload.wikimedia.org/wikipedia/commons/6/62/Latitude_and_Longitude_of_the_Earth.svg" width = 440> <img style="float: right" src="https://upload.wikimedia.org/wikipedia/commons/2/2a/Galactic_coordinates.JPG" width = 440>

### 1. Read in your Data

In [ ]:
# Now we will use astropy to read our data.
import astropy.io.fits as fits

# Read in *.fits data file
Bubbles = fits.open('data/MWbubbles.fits')

# Assign Bubbles_data to contain all the data in this table
Bubbles_data = Bubbles[1].data

# Print the names of each column in this table
print(Bubbles[1].columns.names)

#### Here's a brief description of what each column name refers to:

* MWP = Milky Way Project Catalog ID
* ONames = Other names given each object in the catalog
* GLON = Galactic Longitude
* GLAT = Galactic Latitude
* iXdiam = Inner X Diameter
* iYdiam = Inner Y Diameter
* oXdiam = Outer X Diamter
* Reff = Effective Radius
* Thick = Effective Thickness
* Ecc = Eccentricity
* PA = Ellipse Position Angle
* Hit = Hit Rate (indicates the fraction of people who identified this bubble)
* Disp = Dispersion on the position
* Flag = Hierarchy flags (indicates whether the bubble is associated with other bubbles)

<a href="https://academic.oup.com/mnras/article/424/4/2442/1054276">Science research article with additional information on each of these.</a>

You can explore what type of data is in each of these columns by changing the appropiate line below. 

In [ ]:
# Print the first three and last three of the values in this particular column of the data table
print(Bubbles_data['Reff'])

# The reason 'print' doesn't print all 3744 values to your screen is because that would be annoying.
# Instead 'print' just gives a snapshot (through the first 3 and last 3 values)

### 2. Now let's make a scatter plot of the Bubble's longitude vs latitude

In [ ]:
# Fill in the dashes to create arrays for longitude and latitude
Longitude = Bubbles_data['---']
Latitude = Bubbles_data['---']
errorVal = 0.001 # we are manually setting a fake, flat error for each lat measurement

# Plot the data 
f,ax = plt.subplots(dpi=100)
ax.errorbar(Longitude, Latitude, yerr = errorVal, fmt='ro', markersize=3)

ax.set_xlim(0, 400)
ax.set_ylim(-2, 2)

# Label the plot
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

#### What?! 
Yes, your plot is missing everything in the middle.  Why does the Longitude vs. Latitude plot look so Funny? 

The image below is the area the MilkyWay Project looked at in the disk of our galaxy. 
* Do you notice how Longitude = 0 is at the Galactic Center? 
* Do you notice how the numbers increase to the left (0->60) and decrease to the right (360->300)? 

<img src="http://faculty.wcas.northwestern.edu/aaron-geller/myimages/ssc2014-02a1_ExLrg-v2.jpg" width=1000>

The image below is an artist's representation of the same thing. 
* Looking from our Sun straight towards the Galactic Center means looking at Longitude = 0. 
* Looking to the left means looking from 0->60 degrees and looking to the right means looking from 360->300 degrees.

<img src="http://faculty.wcas.northwestern.edu/aaron-geller/myimages/050816_milky_way_02.jpg" width = "400x" >

### 3. Let's fix our Visualization of the Bubbles' Galactic Latitude and Longitude

To fix your plot of Latitude versus Longitude, let's flip all the Longitude > 180 values so our longitude values decrease smoothly from left to right on our x-axis. <br>

Let's use numpy's where function again.

In [ ]:
#Create a new array that's the same as the original Longitude array
import copy   #import useful library to make a copy of an array
Longitude_new = copy.copy(Longitude)

#Use the 'where' function to identify all the Longitude values > 180
toShift = np.where(Longitude > ---)[0]

#Subtract 360 from all chosen "toShift" Longitude values
Longitude_shifted = Longitude[toShift]- ---

#Replace the old "toShift" Longitude values with these new Longitude values
Longitude_new[toShift] = ---

Now let's plot these new Bubble Longitude vs Latitude values.

In [ ]:
#Plot Longitude_new versus Latitude
f,ax = plt.subplots(figsize=(8,5))
ax.plot(Longitude_new, Latitude,'ro',markersize=3)

#Now play around with which values work best for the plot below
ax.set_xlim(---, ---)
ax.set_ylim(---, ---)

#Label plot
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.show()

### 4. Fit a line to the data.

In [ ]:
# Define the function (i.e., the model) you'll fit to your data
# Here we use a linear fit again (but you have to make the function!)
# Fill in the dashes with the correct input
def linModel(x, ---, ---):
    y = ---
    return(y)

fitCoeff, fitCovar = curve_fit(linModel, Longitude_new, Latitude)
print(f'fit coefficients:\n Slope = {fitCoeff[0]}, Intercept = {fitCoeff[1]}')
y_int = fitCoeff[1]
y_int_err = fitCovar[1][1]

# Plot your data
f,ax = plt.subplots(figsize=(8,5))
#Fill in the dashes with the correct input
ax.errorbar(Longitude_new, Latitude, yerr=errorVal, color='red', marker='.', markersize=3, lw=0)
ax.set_xlim(100,-100)
ax.set_ylim(-1.5,1.5)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Define the x-values for the best-fit line, and find the y-values using our linModel
bestfit_x = np.linspace(---, ---, 200) # x-vals should span the x-axis of our plot!
bestfit_y = linModel(bestfit_x, fitCoeff[---], fitCoeff[---]) #or you can always do *fitCoeff

# Overplot the best-fit line in blue (default color)
# Include a 'zorder=5' argument to make sure the line is plotted over the data.
ax.plot(bestfit_x, bestfit_y, zorder=---, color='k')

 #### Question: Does your result make sense?
 * Why might the best-fit go through Latitude = 0?
 * What does Latitude = 0 coincide with in our galaxy?
 * Let's explore the Latitude bubble distribution further...
 
### 5.  Histogram 
 
Histograms are the natural choice to visualize the number of occurences with respect to a variable (in this case, latitude).  We will use matplotlib's `hist` function to plot this.

In [ ]:
# in addition to plotting the histogram,
# the hist function also returns a tuple of useful information that we will need.
# thus we will make sure we grab the tuple
f,ax = plt.subplots(figsize=(8,5))
n, bins, patches = ax.hist(---, bins=50, facecolor='blue')

ax.set_xlabel('Latitude (deg)')
ax.set_ylabel('# of Bubbles')

As we expected and saw in our previous plot/fit, more bubbles appear near a latitude of 0 degrees.  In fact, the distribution almost appears Gaussian...

Lets attempt to fit a Gaussian to this distribution similar to what we did with the recombination line spectrum.  We will use the tuple returned by `hist` to fit and plot:
    - n: this is the number of occurrences in each bin.
    - bins: this is the value of the bin edges.
    
Notice that `len(bins) = len(n) + 1`, this means we can't plot these together without some modification.  The `bins` array is longer because if each value is the left edge of a bin then there is an additional element for the last right edge. 

Instead, we will create a new array that is the midpoint of each neighboring bin.

In [ ]:
# the first bin is the average of the first two bins
firstBin = (bins[0] + bins[1])/2
# similarily, the last bin is the average of the last two bins
lastBin = (bins[-1] + bins[-2])/2
# the new (50 element) array is then
midBins = np.linspace(---, lastBin, len(---))

# a quick plot to make sure our above modifications made sense
f,ax = plt.subplots(figsize=(8,5))
# it should look identical to the histogram
ax.step(midBins, n, 'k')
ax.set_xlabel('Latitude (deg)')
ax.set_ylabel('# of Bubbles')

In [ ]:
# Redefining the gaussian function (but it's the same as last time!)

def gauss(x, *params):
    amp, center, fwhm = params
    a = ---
    y = ---          
    return y

Now just as we fit the recombination line, you will want to supply an intial guess before utilizing `curve_fit`.

In [ ]:
p0 = np.array([---, ---, ---)

fitCoeffs, fitCov = curve_fit(gauss, ---, ---, p0=p0)

In [ ]:
# another plot to make sure the fit makes sense
f,ax = plt.subplots(figsize=(8,5))
ax.step(---, ---, 'k') # same as how we plotted midBins before

bestfit_x = np.linspace(-1.1, 1.1, 50)  # lets create x-values spanning our histogram plot
ax.plot(bestfit_x, gauss(bestfit_x, *---), 'r')  
# set labels and x limits
plt.show()

In [ ]:
# print out the results
amp = fitCoeffs[---]
center = fitCoeffs[---]
fwhm = fitCoeffs[---]

ampErr = np.sqrt(fitCov[---][---])
centerErr = np.sqrt(fitCov[---][---])
fwhmErr = np.sqrt(fitCov[---][---])

print(f'The largest count is  : {---:5.5} +/-{---:4.2} bubbles')
print(f'The center Latitude is: {---:5.5} +/-{---:4.2} degrees')
print(f'The FWHM is           : {---:5.5} +/-{---:4.2} degrees')

How does the center latitude found with the Gaussian `curve_fit` compare to the intercept found with the line `curve_fit` previously?  Are they within a standard deviation from each other?  Should they be the same?

In [ ]:
print(f'Y-int min: {y_int- ---}')
print(f'Y-int max: {--- + ---}')
print(f'Center min: {center- ---}')
print(f'Center max: {--- + ---}')

### 6. Further Bubble Investigation

In [ ]:
#Identify the arrays with the radius and thickness for all the bubbles
#Note: Reff = effective radius and Thick = effective thickness
radius = Bubbles_data['Reff']
thick = Bubbles_data['Thick']

#### What can we find out about the effective radius?

Use the `max`, `min`, and `mean` functions to print out the results for the data.

In [ ]:
print(f'The maximum radius is: {---}')
print(f'The minimum radius is: {---}')
print(f'The mean radius is: {---}')
print(f'The standard deviation of the radii is: {---}')

#### Identify the longitude/latitude values for the largest bubbles in the catalog

In [ ]:
#Use the 'where' function to identify which bubbles have radii greater than 10.
bigLon = Longitude[---[0]]
bigLat = Latitude[---[0]]

print(f'# of bubbles with radius > 10: {---}')

#Print the Latitude/Longitude values for the bubbles of interest
#Use a 'for' loop to print the values as a pair for each bubble
for i in range(len(bigLon)):
    print(f'Longitude, Latitude: ({---[---]:5.4}, {---[---]:6.3})')

#### How does the bubble thickness relate to the bubble radius?

To finish up our Milky Way bubble investigation, make a plot of the bubble radius vs bubble thickness.  Do you see a trend?  If so, use `curve_fit` to find a best fit curve along with the uncertainties.

In [ ]:
# making an initial plot of the effective radius vs the thickness to see the data
f,ax = plt.subplots()
ax.scatter(---, ---, c='orange', linewidths=0.5, edgecolors='k')

# label the plot
ax.set_xlabel(---)
ax.set_ylabel(---)
plt.show()

In [ ]:
# fitting a linear function
fitCoeffs, fitCov = curve_fit(---, ---, ---)

# plotting the results
f,ax = plt.subplots(figsize=(8,5))
ax.scatter(---, ---, c='orange', linewidths=0.5, edgecolors='k')

bestfit_x = np.linspace(---, ---, ---)  # get x-values from -1 to 12.5
ax.plot(bestfit_x, ---(bestfit_x, *---), 'b')

# set xlim and labels
ax.set_xlabel('Radius')
ax.set_ylabel('Thickness')
plt.show()

In [ ]:
# Printing the results...
slope = fitCoeffs[---]
intercept = fitCoeffs[---]

slopeErr = fitCov[---][---]
interceptErr = fitCov[---][---]

print(f'The slope is    : {slope:5.5} +/-{slopeErr:4.2}')
print(f'The intercept is: {intercept:5.5} +/-{interceptErr:4.2}')

***
<a id='sectionC'></a>
## C. Zooniverse GalaxyZoo Data

### 1. Where are all the galaxies?

Let's see where in the universe our <a href="http://www.galaxyzoo.org"> GalaxyZoo</a> galaxies are located.

You will be using the data file <a href="data/GZ_class_mags.dat">GZ_class_mags.dat</a> which contains all of the classification results for the GalaxyZoo galaxies. Let's see what data is contained in this file. To read in this data file, we will use `ascii.read` command from `astropy`.

In [ ]:
# import needed libraries
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column
from astropy.io import ascii

# read in data table
Galaxies = ascii.read('data/GZ_class_mags.dat')
Galaxies.colnames

In order to determine the <i> location </i> of the galaxies, we will use the galactic latitude and longitude of the galaxies. These are called the <a href="http://astro.unl.edu/classaction/animations/coordsmotion/radecdemo.html">RA and DEC</a> values, and are located in columns three and four. Use these values to create a plot of the location of each galaxy in the sky.

In [ ]:
# First, create two variables that point to the RA and DEC columns
RA = Galaxies[---]
DEC = Galaxies[---]

Lets see where the Galaxies in our table are located by making a RA vs DEC plot.

In [ ]:
f,ax = plt.subplots(figsize=(8,5))
ax.scatter(---, ---, color='k', marker='.', lw=0, alpha=0.5)
ax.set_xlabel('RA')
ax.set_ylabel('Dec')

Your plot of RA vs Dec for all the Galaxy Zoo galaxies will look something like the image below (though doesn't need to be in this same projection). 

The yellow-green squiggle in the image is where the disk of our Milky Way galaxy lives. The disk of our Milky Way blocks our view of distant galaxies in that part of our sky. 

<img src="http://farm2.static.flickr.com/1055/4724975807_79f8722a8d_b.jpg" width = "600x">

### 2. What color are typical spiral galaxies?

Color is typically defined by using the difference of two bands.  In our case, we will subtract a galaxy's i-band (infrared) brightness from its g-band (green) brightness.  This will tell us how much bluer or redder the galaxy is. Smaller values of g-i mean the galaxy is bluer, larger values of g-i means that it's redder (remember small magnitudes means brighter).

In [ ]:
# make a numpy array for the 'g_mag' and 'i_mag' columns each
gmag = np.array(Galaxies[---])
imag = np.array(Galaxies[---])

# now subtract the two arrays to get color
color = --- - ---

# print the first few color values as a test
print(gmag[0:10])
print(imag[0:10])
print(color[0:10])

# print the min and max color
print(f'MIN/MAX color: {min(color)}/{max(color)}')

Notice how the color values for first few galaxies are close to 1 but the min and max color are extremely low and high.

These very high/low values are because there are some sources for which the telescope was unable to get a good value for magnitude or brightness. In the table, the magnitudes for these sources have been set to -9999, as a flag to indicate that they're bad values. 

Often when working with data, you'll need to be aware of 'bad' values and make sure you filter them out.

Lets use `np.where` to find the indices for these bad values.

In [ ]:
# find the indices of the bad values
# make sure you consider both positive OR negative bad values
flag_ind_g = np.where(abs(---) == ---)[0]
flag_ind_i = np.where(abs(---) == ---)[0]

# set the color values for these bad spots to NAN (not a finite #)
color[---] = np.nan
color[---] = np.nan

In [ ]:
# plot a histogram for the galaxy color
f,ax = plt.subplots(figsize=(8,5))

# to avoid the flagged data use: color[np.isfinite(color)] 
ax.hist(---, bins=1000, facecolor='red') # play around with # of bins!
ax.set_xlabel('Color')
ax.set_ylabel('# of Galaxies')
ax.set_xlim(---, ---)

plt.show()

### 3. Does the color depend on the type of galaxy?

Now, lets try to determine whether or not the color has any dependence on the galaxy being a spiral or elliptical type.

In [ ]:
# we will use the np.where function again
# this time, to determine the indices for the two types of galaxies
spiral_ind = np.where(Galaxies['Spiral'] == 1)[0] # These data were saved as booleans: 1 means yes, spiral!
elliptical_ind = np.where(Galaxies['Elliptical'] == 1)[0] # 1 means yes, elliptical!

# For fun: what does it mean if a galaxy has a '0' in both columns?

# make a corresponding array that picks out these indices
spiral_color = color[---]
elliptical_color = color[---]

In [ ]:
# plot a histogram for the spiral galaxies
f,ax = plt.subplots(figsize=(8,5))
plt.hist(---[---], bins=1000) #I suggest to add a condition to filter out infinite values in spiral_color
ax.set_xlabel('Color')
ax.set_ylabel('# of Spiral Galaxies')
ax.set_xlim(0,2.5)

In [ ]:
# do the same but for the elliptical galaxies
f,ax = plt.subplots(figsize=(8,5))
plt.hist(---[---], bins=1000)
ax.set_xlabel('Color')
ax.set_ylabel('# of Spiral Galaxies')
ax.set_xlim(0,2.5)

Compare the two plots.  Do you notice any difference?  Does one type of galaxy appear to be more red than the other?  Remember, the lower the value for color (g-i), the bluer the galaxy.  The bluer color indicates that the galaxy is currently making new star (i.e., has active star formation).

Lets plot both together to further illustrate the difference

In [ ]:
f,ax = plt.subplots(figsize=(8,5))

# give the bluer type a 'blue' color and the redder type a...
# also give the plots some transparency (using 'alpha') and a 'label'
plt.hist(elliptical_color[---], bins=500, alpha=0.5, facecolor='red', label='Elliptical')
plt.hist(spiral_color[---], bins=500, alpha=0.5, facecolor='blue', label='Spiral')
ax.set_xlabel('Color')
ax.set_ylabel('# of Galaxies')
ax.set_xlim(0,2.5)

# add in a legend
ax.legend() #tip: add frameon=False to the ax.legend() command to remove the outline from around the legend

As we see, spiral galaxies tend to be bluer than elliptical.  Is this consistent with a blue color indicating current star formation?

However, also notice the exceptions. In your histogram, you see that a number of spiral galaxies are redder in color (have higher 'g-i' color values). These are galaxies where classifiers saw they had spiral arms, but their color tells us that they have no active star formation happening. <a href="http://www.sciencedaily.com/releases/2008/11/081124194936.htm">A special process has shut off star formation in these galaxies.</a> 

Finding this intriguing result was possible because of the huge numbers of galaxies in GalaxyZoo!

### 4. Examining an individual galaxy.

If you identify a subset of galaxies in the GalaxyZoo data table that you'd like to look at individually in more detail, you can do the following:

In [ ]:
# query particular galaxies of interest, for example, red spirals
# note still need to avoid NAN values

redSpirals = np.where((Galaxies['Spiral'][np.isfinite(color)] == 1) & \
                      (color[np.isfinite(color)] < 1.91) & \
                      (color[np.isfinite(color)] > 1.9))[0]

print(f'# of red spirals = {len(redSpirals)}')

# print out where these are located
for i in redSpirals:
    print(f'RA, DEC = {RA[i]:7.5}, {DEC[i]:7.5}')

#### Visualize these red spirals.

Now copy and paste these RA,Dec values into the <a href="http://skyserver.sdss.org/dr7/en/tools/chart/navi.asp">SDSS SkyServer finder chart</a>. You'll see an image of the galaxy of interest. <br>

Click on 'Quick Look' in the middle right of that page to get additional information about your galaxy.<br>

Do you see how they're red spirals?! Check out <a href="http://arxiv.org/abs/0910.4113">Dr. Karen Masters article on Red Spirals in Galaxy Zoo</a>.

***
<a id='sectionD'></a>
## D. Random Numbers

First let's talk about drawing values randomly in general.

Computers don't know how to draw truly random numbers. Instead they generate some large sequence of numbers that appear random, but are really deterministic. Usually you have to define a "seed" value to define where to start the sequence.

In python you don't actually have to define the "seed" value explicitly, but you can in order to keep your "random" draw the same each time you run the code.  

In general, we will use the numpy module to generate random numbers.

In [ ]:
#Defining the "seed" for numpy's random function
np.random.seed(seed = 1234567) #can do any number. many people choose 42.
# the point of defining "seed" means that it may be a "random" number, but you'll get the same 
# random number each time, meaning you can verify that the same set of code will do 
# the exact same thing each iteration (and plots will look identical).

# In principle though you can change this value and/or leave it blank
# to see the code work on other random numbers!

Now, let's draw randomly from a Gaussian (aka 'Normal') distribution. <br>

In [ ]:
#Numpy also has a built-in function to draw random numbers from a Gaussian
#define the Gaussian using the mean, standard deviation (stdev), 
#  and choose some number of values (Nvals) that you want to draw
mean = ---
stdev = ---
Nvals = 100 # try changing this value and running the next few cells for a few different values
rg = np.random.normal(mean, stdev, size = Nvals)

#print the mean, min and max of this list of random mumbers
print(f'mean,min,max of the random sample: {np.mean(rg)},{min(rg)},{max(rg)}')

In [ ]:
#plot a histogram of these values
f,ax = plt.subplots(figsize=(5,3))
n, bins, patches = ax.hist(rg, 100, color = 'b', alpha = 0.6)

#overplot a dashed vertical line that shows the mean value
#tip: can use axvline to plot a vertical line:
ax.axvline(np.mean(rg), color='r', linestyle='--', lw=1.5)

ax.set_xlabel("value")
ax.set_ylabel("N")

In [ ]:
#How many values are within 1,2,3 sigma of the mean for this numpy.random.normal function?
#Hint: you can use numpy.where to find all the values that are within some range of the mean

for fac in [1,2,3]:
    ntest = float(len(np.where( abs(rg - ---) < fac*---)[0]))
    print(f'    percent inside of {fac} times sigma = {ntest/float(Nvals)*100}')

#### Question: Do the numbers for sigma match up to the <a href="http://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule">right answer for a gaussian distribution</a>?

If not exactly, why not? What could you change to get closer to the right answer? Make this change above, and see if you can come closer to the correct values.

#### What if you didn't have this easy np.random.normal function? You could instead sample a Gaussian using this "Box-Muller" procedure with two random numbers between 0 and 1. 

In python you can draw numbers from a uniform distribution with np.random.random()
This method is very useful, e.g., in C or fortran (and many other codes)

In [ ]:
#
rg_bm = []
for i in range(Nvals):
#draw your first random number
    x1 = np.random.random()
#draw your second random number    
    x2 = np.random.random()
#just in case we drew 0
    while (x1 == 0): x1 = np.random.random()
    while (x2 == 0): x2 = np.random.random()
#this draws from a Guassian distribution
    rg_bm.append(stdev * (-2.*np.log(x1))**0.5 * np.cos(2.*np.pi*x2) + mean)
    
rg_bm = np.array(rg_bm) #just to keep them in the same format

In [ ]:
#How many values are within 1,2,3 sigma for the "Box-Muller" procedure?
print(f'Box-Muller :')
for fac in [1,2,3]:
    ntest = float(len(np.where( abs(rg_bm - mean) < fac*stdev)[0]))
    print(f'    percent inside of {fac} times sigma = {ntest/float(Nvals)*100}')

In [ ]:
#plot a histogram of these values
f,ax = plt.subplots(figsize=(5,3))
#first the numpy.random.normal results
n, bins, patches = ax.hist(rg, 50, color = 'b', alpha = 0.5)#, range = [mean - 5.*stdev, mean + 5.*stdev])
#now the Box-Muller results
nb, binsb, patchesb = ax.hist(rg_bm, 50, color = 'g', alpha = 0.5)#, range = [mean - 5.*stdev, mean + 5.*stdev])

#overplot a dashed line that shows the mean value
ax.axvline(np.mean(rg_bm), color='r', linestyle='--', lw=2)

#overplot dotted lines that show the 1,2,3 sigma values
for fac in [1,2,3]:
    ax.plot([mean + fac*stdev, mean + fac*stdev],[0,max(n)], 'r:', linewidth = 1.5)
    ax.plot([mean - fac*stdev, mean - fac*stdev],[0,max(n)], 'r:', linewidth = 1.5)

    
ax.set_xlabel("value")
ax.set_ylabel("N")
ax.set_xlim(0,4)

# Congratulations! You've completed this workshop!